In [1]:
import re
import pandas as pd
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

**공백 및 특수문자 처리 함수**

In [ ]:
def preprocess_sentence(text):
    """
    입력된 텍스트에서 공백과 특수문자를 제거하는 함수
    - 불필요한 공백 제거
    - 특수문자 및 숫자 제거
    """
    # 단어와 구두점(punctuation) 사이 공백
    #text = re.sub(r"([?.!,])", r" \1 ", text)
    #text = re.sub(r'[" "]+', " ", text)
    # 특수문자 및 숫자 제거
    #text = re.sub('^가-힣ㄱ-하-ㅣa-zA-Z.', '', text)  # 한글, 공백만 남기고 제거
    # 불필요한 공백 제거
    #text = re.sub(r'\s+', ' ', text).strip()  # 여러 공백을 하나로 치환 후 양쪽 공백 제거
    return text

**데이터 읽어오기**

In [2]:
import pandas as pd

In [3]:
train_file_path = '../aiffel/train.csv'
test_file_path = '../aiffel/test.csv'
normal_file_path = '../aiffel/normal_data.csv'

In [4]:
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
normal = pd.read_csv(normal_file_path, nrows=1000) # 클래스 균형

In [5]:
print('train_df 전체 샘플 수 :', len(train_df))
print('test_df 전체 샘플 수 :', len(test_df))
print('normal 전체 샘플 수 :', len(normal))

train_df 전체 샘플 수 : 3950
test_df 전체 샘플 수 : 500
normal 전체 샘플 수 : 1000


데이터 확인

In [ ]:
train_df.head()

In [ ]:
normal.head()

In [ ]:
test_df.head()

훈련 데이터 - 라벨 정수 인코딩, 인풋 데이터 전처리

In [6]:
label_map = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}

In [7]:
train_df['encoded_label'] = train_df['class'].map(label_map)

print(train_df['encoded_label'].unique())
train_df.tail()

[0 3 1 2]


,idx,class,conversation,encoded_label
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...,3
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,1
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,2
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...,1
3949,3949,직장 내 괴롭힘 대화,희정씨\n네?\n주말에 시간이 넘쳐나나봐\n갑자기 왜그러세요?\n손이 빤짝빤짝 네일...,2


In [8]:
normal = normal.rename(columns={'0': 'conversation'})
normal['encoded_label'] = 4
normal.head()

,conversation,encoded_label
0,지금 배달되나요?\n아 네 배달됩니,4
1,짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?\n특해물 짬뽕도 있고 전복 새우...,4
2,전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?\n전복 짬뽕 시키면 전복이 들어가죠\...,4
3,여기 #주소#인데 배달되나요?\n#주소#는 안됩니,4
4,중국집 명성루죠? 배달 지금 가능한가요?\n예 배달 가능합니,4


In [9]:
train_df = pd.concat([train_df, normal], axis=0, ignore_index=True)
train_df.tail()

,idx,class,conversation,encoded_label
4945,NaN,NaN,여기 주차장에 주차하는 거 맞아요?\n네 5000원 이상만 하시면 2시간 찍어드려,4
4946,NaN,NaN,이거 새로 나온 거예요?\n새로 나온 건 핫크리스피라고 해요. 크리스피버거는 닭다리...,4
4947,NaN,NaN,뻑뻑하진 않나요 고기가?\n닭가슴살이라 조금 퍽퍽해요\n근데 칼로리가 좀 낮겠네요\n,4
4948,NaN,NaN,어떤 게 좀 인기 있어요?\n이게 제일 무난하게 잘 나가구요 저쪽은 치킨 종류 찾으...,4
4949,NaN,NaN,그럼 이 불고기요 버거 셋트인가요 이게?\n이쪽은 세트고 이쪽은 버거인데요 지금 세...,4


In [10]:
# 필요없는 컬럼 삭제 후 데이터 셔플
train_df = train_df.drop(columns=['idx', 'class'])
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_df.head()

,conversation,encoded_label
0,친구야 저번에 돈 빌려줘서 너무 고마웠어\n아. 응.\n근데 내가 오늘도 깜빡하고 ...,1
1,어떤 게 잘 나가요?\n이것도 취향 따라서 가는데 위트 잘 나가고,4
2,야 나 5만 원 빌려줘라 \n전에도 내가 3만 원 빌려줬잖아.\n야 내가 안 준대?...,1
3,무릎 꿇고 손 들어.\n알겠습니다.\n지금부터 제대로 대답하지 않으면 인질을 한 명...,0
4,내가 싯팔 영화 스포하지 말랬지\n.\n오징어게임 아직 보지도 않았는데 비밀을 알아...,0


In [12]:
X_train = train_df['conversation']
y_train = train_df['encoded_label']

In [13]:
X_train.head()

0    친구야 저번에 돈 빌려줘서 너무 고마웠어\n아. 응.\n근데 내가 오늘도 깜빡하고 ...
1                 어떤 게 잘 나가요?\n이것도 취향 따라서 가는데 위트 잘 나가고
2    야 나 5만 원 빌려줘라 \n전에도 내가 3만 원 빌려줬잖아.\n야 내가 안 준대?...
3    무릎 꿇고 손 들어.\n알겠습니다.\n지금부터 제대로 대답하지 않으면 인질을 한 명...
4    내가 싯팔 영화 스포하지 말랬지\n.\n오징어게임 아직 보지도 않았는데 비밀을 알아...
Name: conversation, dtype: object

In [14]:
y_train.head()

0    1
1    4
2    1
3    0
4    0
Name: encoded_label, dtype: int64

테스트 데이터 - 전처리

In [15]:
X_test = test_df['text']

In [16]:
X_test[:5]

0    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1    우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...
2    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4    아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
Name: text, dtype: object

# TFBertForSequenceClassification
- Bert가 텍스트 분류 문제에서 좋은 성능을 보인다고 해서 선택했다.
- 하지만, [데이터에 비해 모델 사이즈가 큰 탓인지] 학습이 제대로 되지 않는 문제가 발생한다.
- 비슷한 아키텍쳐의 조금 더 작은 사이즈의 모델을 사용해봐야겠다.
- 이후에 제대로 결과가 나오지 않는 이유를 모색해볼 예정이다.

**토크나이징**

In [23]:
import numpy as np
from transformers import BertTokenizer

In [18]:
# BERT 모델의 토크나이저를 로드 (한국어 BERT 모델)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [19]:
batch_size = 32
max_length = 330

In [21]:
def tokenize_dialogues(dialogues, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []
    
    for dialogue in dialogues:
        # 각 대화 세트를 하나의 긴 문장으로 합침
        dialogue_text = " [SEP] ".join(dialogue)  # 대화 턴들을 [SEP]로 구분
        
        # 토크나이징
        encoded = tokenizer(
            dialogue_text, 
            truncation=True, 
            padding='max_length', 
            max_length=max_length, 
            return_tensors='tf'
        )
        
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    # 텐서 형태로 변환
    input_ids = tf.convert_to_tensor(np.array(input_ids))
    attention_masks = tf.convert_to_tensor(np.array(attention_masks))
    
    return input_ids, attention_masks

In [24]:
# 대화 데이터를 토크나이징
X_train_input_ids, X_train_attention_masks = tokenize_dialogues(X_train, tokenizer, max_length)

In [ ]:
# BERT 토크나이저로 토크나이즈
#tokenized_X_train = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

#print(tokenized_X_train['input_ids'])
#print(tokenized_X_train['attention_mask'])

In [ ]:
#tokenized_X_test = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

#print(tokenized_X_test['input_ids'])
#print(tokenized_X_test['attention_mask'])

In [25]:
from sklearn.model_selection import train_test_split

In [ ]:
# 'input_ids', 'attention_mask', 'token_type_ids'를 NumPy 배열로 변환
#X_train_input_ids = np.array(tokenized_X_train['input_ids'])
#X_train_attention_mask = np.array(tokenized_X_train['attention_mask'])
#X_train_token_type_ids = np.array(tokenized_X_train['token_type_ids'])

#X_test_input_ids = np.array(tokenized_X_test['input_ids'])
#X_test_attention_mask = np.array(tokenized_X_test['attention_mask'])
#X_test_token_type_ids = np.array(tokenized_X_test['token_type_ids'])

In [27]:
# train/val split
X_train_enc, X_val_enc, y_train_split, y_val_split = train_test_split(
    np.array(X_train_input_ids), y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [29]:
# X_train_enc와 X_val_enc에 해당하는 attention_mask, token_type_ids를 정확히 분리
X_train_encodings = {
    'input_ids': X_train_enc,
    'attention_mask': [X_train_attention_masks[i] for i in range(len(X_train_enc))],
    #'token_type_ids': [X_train_token_type_ids[i] for i in range(len(X_train_enc))]
}

X_val_encodings = {
    'input_ids': X_val_enc,
    'attention_mask': [X_train_attention_masks[i] for i in range(len(X_val_enc))],
    #'token_type_ids': [X_train_token_type_ids[i] for i in range(len(X_val_enc))]
}

# 테스트 데이터셋의 인코딩
'''X_test_encodings = {
    'input_ids': X_test_input_ids,
    'attention_mask': X_test_attention_mask,
    #'token_type_ids': X_test_token_type_ids
}'''

"X_test_encodings = {\n    'input_ids': X_test_input_ids,\n    'attention_mask': X_test_attention_mask,\n    #'token_type_ids': X_test_token_type_ids\n}"

TensorFlow Dataset 준비

In [30]:
# 훈련용 데이터셋 (라벨 포함)
def create_tf_dataset(encodings, labels=None):
    if labels is not None:
        dataset = tf.data.Dataset.from_tensor_slices((
            dict(encodings),
            labels
        ))
    else:
        dataset = tf.data.Dataset.from_tensor_slices(dict(encodings))  # 라벨 없이 데이터만
    return dataset

In [31]:
# 훈련 데이터셋
train_dataset = create_tf_dataset(X_train_encodings, y_train_split).batch(4)

# 검증 데이터셋
val_dataset = create_tf_dataset(X_val_encodings, y_val_split).batch(4)

# 테스트 데이터셋 (라벨 없이)
#test_dataset = create_tf_dataset(X_test_encodings).batch(4)

**모델 준비**

In [32]:
from transformers import TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

In [33]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=5)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
model.compile(optimizer=Adam(learning_rate=5e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**모델 학습**

In [35]:
model.fit(train_dataset, epochs=3, validation_data=val_dataset)

Epoch 1/3


ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py:1443 call  *
        outputs = self.bert(
    /opt/conda/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py:645 call  *
        embedding_output = self.embeddings(
    /opt/conda/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py:202 call  *
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
    /opt/conda/lib/python3.9/site-packages/keras/engine/base_layer.py:1037 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/layers/normalization/layer_normalization.py:285 call
        scale, offset = _broadcast(self.gamma), _broadcast(self.beta)
    /opt/conda/lib/python3.9/site-packages/keras/layers/normalization/layer_normalizat

**모델 예측**

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
# 예측된 logits 추출
logits = predictions.logits
logits

In [ ]:
# logits -> 예측된 클래스 (가장 높은 확률을 가진 클래스를 선택)
pred_labels = np.argmax(logits, axis=-1)
pred_labels

# ALBERT

In [ ]:
from transformers import TFAlbertForSequenceClassification, AlbertTokenizer

In [ ]:
tokenizer_Albert = AlbertTokenizer.from_pretrained('albert-base-v2')

In [ ]:
# Albert 토크나이저로 토크나이즈
tokenized_X_train = tokenizer_Albert(X_train.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

print(tokenized_X_train)

In [ ]:
tokenized_X_test = tokenizer_Albert(X_test.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

print(tokenized_X_test)

In [ ]:
# 'input_ids', 'attention_mask', 'token_type_ids'를 NumPy 배열로 변환
X_train_input_ids = np.array(tokenized_X_train['input_ids'])
X_train_attention_mask = np.array(tokenized_X_train['attention_mask'])
#X_train_token_type_ids = np.array(tokenized_X_train['token_type_ids'])

X_test_input_ids = np.array(tokenized_X_test['input_ids'])
X_test_attention_mask = np.array(tokenized_X_test['attention_mask'])
#X_test_token_type_ids = np.array(tokenized_X_test['token_type_ids'])

In [ ]:
# train/val split
X_train_enc, X_val_enc, y_train_split, y_val_split = train_test_split(
    X_train_input_ids, y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [ ]:
# X_train_enc와 X_val_enc에 해당하는 attention_mask, token_type_ids를 정확히 분리
X_train_encodings = {
    'input_ids': X_train_enc,
    'attention_mask': [X_train_attention_mask[i] for i in range(len(X_train_enc))],
    #'token_type_ids': [X_train_token_type_ids[i] for i in range(len(X_train_enc))]
}

X_val_encodings = {
    'input_ids': X_val_enc,
    'attention_mask': [X_train_attention_mask[i] for i in range(len(X_val_enc))],
    #'token_type_ids': [X_train_token_type_ids[i] for i in range(len(X_val_enc))]
}

# 테스트 데이터셋의 인코딩
X_test_encodings = {
    'input_ids': X_test_input_ids,
    'attention_mask': X_test_attention_mask,
    #'token_type_ids': X_test_token_type_ids
}

In [ ]:
# 훈련 데이터셋
train_dataset = create_tf_dataset(X_train_encodings, y_train_split).batch(16)

# 검증 데이터셋
val_dataset = create_tf_dataset(X_val_encodings, y_val_split).batch(16)

# 테스트 데이터셋 (라벨 없이)
test_dataset = create_tf_dataset(X_test_encodings).batch(16)

In [ ]:
model_Albert = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=5)

In [ ]:
model_Albert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_Albert.fit(train_dataset, epochs=3, validation_data=val_dataset, batch_size=16)

In [ ]:
predictions = model_Albert.predict(test_dataset)

In [ ]:
# 예측된 logits 추출
logits = predictions.logits
logits

In [ ]:
# logits -> 예측된 클래스 (가장 높은 확률을 가진 클래스를 선택)
pred_labels = np.argmax(logits, axis=-1)
pred_labels

# GPT-2

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer_Auto = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')

In [ ]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def convert_examples_to_features(sent_list, max_seq_len, tokenizer):
    input_ids = []

    for conversation in tqdm(sent_list, total=len(sent_list)):
        bos_token = [tokenizer.bos_token]  # 시작 토큰
        eos_token = [tokenizer.eos_token]  # 종료 토큰
        unused_token = ['<unused0>']  # 마지막에 추가할 토큰

        # '\n'으로 문장들 분리
        sentences = conversation.split("\n")

        tokens = []  # 전체 토큰 리스트

        for i, sentence in enumerate(sentences):
            # 각 문장에 <s>와 </s> 추가
            sentence_tokens = bos_token + tokenizer.tokenize(sentence) + eos_token
            tokens += sentence_tokens  # 문장 뒤에 <s> 추가

        # 마지막 문장 뒤에 <unused0> 추가
        tokens += unused_token  # 마지막에 <unused0> 추가
        tokens += eos_token  # 마지막에 종료 토큰 추가

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        # 패딩 추가
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        # 길이 확인
        assert len(input_id) == max_seq_len, f"Error with input length {len(input_id)} vs {max_seq_len}"

        # 결과 저장
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids


In [ ]:
converted_X_train = convert_examples_to_features(X_train, max_seq_len=max_length, tokenizer=tokenizer_Auto)
converted_X_test = convert_examples_to_features(X_test, max_seq_len=max_length, tokenizer=tokenizer_Auto)

In [ ]:
# train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X_train_input_ids, y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [ ]:
from transformers import TFGPT2Model

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model_GPT2 = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=5)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=4, validation_data = (X_val, y_val)
)

# hgf

In [50]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")

In [ ]:
max_length = 330

In [39]:
# BERT 토크나이저로 토크나이즈
tokenized_X_train = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

print(tokenized_X_train)

{'input_ids': <tf.Tensor: shape=(4950, 330), dtype=int32, numpy=
array([[    2,  9466,  5059, ...,     0,     0,     0],
       [    2,  8879,  1977, ...,     0,     0,     0],
       [    2,  3545,  2239, ...,     0,     0,     0],
       ...,
       [    2, 10412, 10256, ...,     0,     0,     0],
       [    2, 16647,  3805, ...,     0,     0,     0],
       [    2,  2239, 16835, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4950, 330), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4950, 330), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=in

In [41]:
tokenized_X_test = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

print(tokenized_X_test)

{'input_ids': <tf.Tensor: shape=(500, 330), dtype=int32, numpy=
array([[    2, 17613,  5563, ...,     0,     0,     0],
       [    2,  8472,  5334, ...,     0,     0,     0],
       [    2,  2276,  9062, ...,     0,     0,     0],
       ...,
       [    2,  3545,  2276, ...,     0,     0,     0],
       [    2,  3545,  2276, ...,     0,     0,     0],
       [    2,  9686,  3805, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(500, 330), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(500, 330), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [42]:
# 'input_ids', 'attention_mask', 'token_type_ids'를 NumPy 배열로 변환
X_train_input_ids = np.array(tokenized_X_train['input_ids'])
X_train_attention_mask = np.array(tokenized_X_train['attention_mask'])
X_train_token_type_ids = np.array(tokenized_X_train['token_type_ids'])

X_test_input_ids = np.array(tokenized_X_test['input_ids'])
X_test_attention_mask = np.array(tokenized_X_test['attention_mask'])
X_test_token_type_ids = np.array(tokenized_X_test['token_type_ids'])

In [43]:
# train/val split
X_train_enc, X_val_enc, y_train_split, y_val_split = train_test_split(
    X_train_input_ids, y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [44]:
# X_train_enc와 X_val_enc에 해당하는 attention_mask, token_type_ids를 정확히 분리
X_train_encodings = {
    'input_ids': X_train_enc,
    'attention_mask': [X_train_attention_masks[i] for i in range(len(X_train_enc))],
    #'token_type_ids': [X_train_token_type_ids[i] for i in range(len(X_train_enc))]
}

X_val_encodings = {
    'input_ids': X_val_enc,
    'attention_mask': [X_train_attention_masks[i] for i in range(len(X_val_enc))],
    #'token_type_ids': [X_train_token_type_ids[i] for i in range(len(X_val_enc))]
}

# 테스트 데이터셋의 인코딩
X_test_encodings = {
    'input_ids': X_test_input_ids,
    'attention_mask': X_test_attention_mask,
    #'token_type_ids': X_test_token_type_ids
}

In [45]:
# 훈련용 데이터셋 (라벨 포함)
def create_tf_dataset(encodings, labels=None):
    if labels is not None:
        dataset = tf.data.Dataset.from_tensor_slices((
            dict(encodings),
            labels
        ))
    else:
        dataset = tf.data.Dataset.from_tensor_slices(dict(encodings))  # 라벨 없이 데이터만
    return dataset

In [46]:
# 훈련 데이터셋
train_dataset = create_tf_dataset(X_train_encodings, y_train_split).batch(4)

# 검증 데이터셋
val_dataset = create_tf_dataset(X_val_encodings, y_val_split).batch(4)

# 테스트 데이터셋 (라벨 없이)
test_dataset = create_tf_dataset(X_test_encodings).batch(4)

In [52]:
model = TFAutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

404 Client Error: Not Found for url: https://huggingface.co/snunlp/KR-FinBert-SC/resolve/main/tf_model.h5


OSError: Can't load weights for 'snunlp/KR-FinBert-SC'. Make sure that:

- 'snunlp/KR-FinBert-SC' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'snunlp/KR-FinBert-SC' is the correct path to a directory containing a file named one of tf_model.h5, pytorch_model.bin.



**데이터 전처리에 문제 있는 것이 맞는듯**

---
# 내보내기

**예측 저장**

In [ ]:
# 예측된 라벨을 test_df에 컬럼으로 추가
test_df['class'] = pred_labels

test_df.tail()

In [ ]:
test_df = test_df.drop(columns=['text'])

test_df.tail()

In [ ]:
# 결과를 submission.csv로 저장 (index는 제외)
test_df.to_csv('submission.csv', index=False)